# Create a manifest file

The *manifest.txt* file created here is intended to be used with [gdc-client](https://gdc.cancer.gov/access-data/gdc-data-transfer-tool)

Please refer to [gdc API](https://docs.gdc.cancer.gov/API/Users_Guide/Getting_Started/) for reference about query done here

In [1]:
%load_ext watermark
%watermark -v -m  -u -n -p pandas,requests,json -a Filippo_Valle -g -r -b -w

Filippo_Valle 
last updated: Fri Jul 10 2020 

CPython 3.7.6
IPython 7.15.0

pandas 1.0.4
requests 2.23.0
json 2.0.9

compiler   : GCC 7.5.0
system     : Linux
release    : 4.19.76-linuxkit
machine    : x86_64
processor  : x86_64
CPU cores  : 2
interpreter: 64bit
Git hash   : 251fb0f8164450c4e383df3cbf21951a4a5a7654
Git repo   : git@github.com:fvalle1/cancers
Git branch : develop
watermark 2.0.2


In [ ]:
import requests
import json
import pandas as pd

In [ ]:
# use file endopint
files_endpt = "https://api.gdc.cancer.gov/files"

# The 'fields' parameter is passed as a comma-separated string of single names.
fields = [
    "file_name",
    #"analysis.workflow_type",
    #"experimental_strategy",
    "cases.project.primary_site",
    "cases.project.disease_type",
    "cases.project.project_id",
    "cases.submitter_id",
    "cases.samples.portions.analytes.aliquots.submitter_id",
    "cases.demographic.vital_status",
    "cases.demographic.days_to_birth",
    "cases.demographic.days_to_death",
    "cases.demographic.gender",
    "cases.diagnoses.primary_diagnosis",
    "cases.diagnoses.tumor_stage",
    "cases.diagnoses.tumor_grade",
    "cases.diagnoses.last_known_disease_status",
    "cases.diagnoses.progression_or_recurrence"
    "cases.diagnoses.days_to_last_follow_up",
    "cases.diagnoses.age_at_diagnosis",
    "cases.diagnoses.morphology",
    "cases.diagnoses.tissue_or_organ_of_origin",
    "cases.exposures.years_smoked",
    "cases.exposures.cigarettes_per_day",
    "cases.samples.longest_dimension",
    "samples.portions.analytes.aliquots.submitter_id",
]

fields = ','.join(fields)

Filter gene expression data obtained with *RNA-Seq HTSeq* in txt format either from **BRCA** or **LUAD** and **LUSC**

Don't forget to select the correct filter for *cases.project.project_id*

In [ ]:
# Tissue project filters
filters = {
    "op": "and",
    "content":[
        {
        "op": "in",
        "content":{
            "field": "files.data_type",
            "value": ["Gene Expression Quantification"]
            }
        },
        {
        "op": "in",
        "content":{
               "field": "files.analysis.workflow_type",
                "value": ["HTSeq - FPKM"]
                }
        },
        {
        "op": "in",
        "content":{
            "field": "files.data_format",
            "value": ["TXT"]
            }
        },
        {
        "op": "in",
        "content":{
            "field": "cases.project.project_id",
            "value": ['TCGA-LUAD', 'TCGA-LUSC'] #<-- uncomment this line for Lung
            #"value": ["TCGA-BRCA"] #<-- uncomment this line for Breast
            }
        }
        
    ]
}

In [ ]:
# A POST is used, so the filter parameters can be passed directly as a Dict object.
##remove manifest to have file fields
params = {
   "return_type": "manifest",
    "filters": filters,
    "fields": fields,
    "format": "TSV",
    "size": "15000"
    }

In [ ]:
# The parameters are passed to 'json' rather than 'params' in this case
response = requests.post(files_endpt, headers = {"Content-Type": "application/json"}, json = params)

#print(response.content.decode("utf-8"))

In [ ]:
with open("manifest.txt","w") as manifest:
    manifest.write(response.content.decode("utf-8"))

# Files

Write a file with all metadata, useful in next analyses

In [ ]:
params = {
    "filters": json.dumps(filters),
    "fields": fields,
    "format": "TSV",
    "size": "15000"
    }
response = requests.post(files_endpt, headers = {"Content-Type": "application/json"}, json = params)
with open("files.txt","w") as files:
    files.write(response.content.decode("utf-8"))

In [ ]:
df_files = pd.read_csv("files.txt", sep='\t')
df_files.drop("id", axis=1, inplace=True)
df_files.set_index("file_name", inplace=True)
df_files.head(3)

In [ ]:
# save file to dat
df_files.to_csv("files_tcga.dat", header=True)